In [1]:
import os
os.chdir('C:/workspace/Paper/ComputerVision/ResNets')
print(os.getcwd())

C:\workspace\Paper\ComputerVision\ResNets


In [2]:
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from resnet import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152

import os

In [3]:
def lr_scheduler(optimizer, epoch):

    lr = learning_rate
    if epoch >= 50:
        lr /= 10

    if epoch >= 100:
        lr /= 10

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def init_weight(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform(m.weight)
        m.bias.data.fill_(0.01)

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding = 4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

transform_test = transforms.Compose([
    transforms.ToTensor()
])

train_dataset = torchvision.datasets.CIFAR10(root = 'E:/workspace', train = True, download = True, transform = transform_train)
test_dataset = torchvision.datasets.CIFAR10(root = 'E:/workspace', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 256, shuffle = True, num_workers = 8)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 256, shuffle = False, num_workers = 8)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = ResNet50()

model.apply(init_weight)
model = model.to(device)

<ipython-input-3-e84569abd4d2>:15: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


In [6]:
torch.cuda.is_available()

True

In [11]:
learning_rate = .1
num_epoch = 100
model_name = 'ResNets_best_model_230115.pth'

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum = .9, weight_decay = .0001)

train_loss = 0
valid_loss = 0
correct = 0
total_cnt = 0
best_acc = 0

In [14]:
for epoch in range(num_epoch):
    print(f"====== { epoch+1} epoch of { num_epoch } ======")

    model.train()
    lr_scheduler(optimizer, epoch)

    train_loss = 0
    valid_loss = 0
    correct = 0
    total_cnt = 0

    for step, batch in enumerate(train_loader):
        batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
        optimizer.zero_grad()
        
        logits = model(batch[0])
        loss = loss_fn(logits, batch[1])
        loss.backward()
        
        optimizer.step()
        train_loss += loss.item()
        _, predict = logits.max(1)
        
        total_cnt += batch[1].size(0)
        correct +=  predict.eq(batch[1]).sum().item()
        
        if step % 100 == 0 and step != 0:
            print(f"\n====== { step } Step of { len(train_loader) } ======")
            print(f"Train Acc : { correct / total_cnt }")
            print(f"Train Loss : { loss.item() / batch[1].size(0) }")
            
    correct = 0
    total_cnt = 0

    # test
    with torch.no_grad():
        model.eval()
        for step, batch in enumerate(test_loader):
            batch[0], batch[1] = batch[0].to(device), batch[1].to(device)
            total_cnt += batch[1].size(0)
            outputs = model(batch[0])

            valid_loss += loss_fn(outputs, batch[1])
            _, predict = outputs.max(1)
            correct += predict.eq(batch[1]).sum().item()

        valid_acc = correct / total_cnt

        print(f"Valid Acc : {valid_acc}")
        print(f"Valid Loss : {valid_loss / total_cnt}")


        if (valid_acc > best_acc):
            best_acc = valid_acc
            torch.save(model, model_name)
            print("model saved!")

====== 1 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.2040145420792079
Train Loss : 0.007674338761717081
Valid Acc : 0.1941
Valid Loss : 0.024771520867943764
model saved!
====== 2 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.24388923267326731
Train Loss : 0.007845763117074966
Valid Acc : 0.1966
Valid Loss : 0.01206472609192133
model saved!
====== 3 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.28194616336633666
Train Loss : 0.0073771667666733265
Valid Acc : 0.3314
Valid Loss : 0.008094390854239464
model saved!
====== 4 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.3300974628712871
Train Loss : 0.007262491155415773
Valid Acc : 0.368
Valid Loss : 0.007225011475384235
model saved!
====== 5 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.3610380569306931
Train Loss : 0.006831890903413296
Valid Acc : 0.3961
Valid Loss : 0.007459290325641632
model saved!
====== 6 epoch of 100 ======

====== 100 Ste

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000295D63E78B0>
Traceback (most recent call last):
  File "c:\Users\LEE\anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "c:\Users\LEE\anaconda3\lib\site-packages\torch\utils\data\dataloader.py", line 1295, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Valid Acc : 0.6557
Valid Loss : 0.0039008907042443752
model saved!
====== 50 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.6620126856435643
Train Loss : 0.0036774366162717342
Valid Acc : 0.635
Valid Loss : 0.00410860450938344
====== 51 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.6888923267326733
Train Loss : 0.0034130928106606007
Valid Acc : 0.6865
Valid Loss : 0.0035223537124693394
model saved!
====== 52 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.6987159653465347
Train Loss : 0.003620314411818981
Valid Acc : 0.6942
Valid Loss : 0.003463817061856389
model saved!
====== 53 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.7032410272277227
Train Loss : 0.0035134130157530308
Valid Acc : 0.6969
Valid Loss : 0.003454542485997081
model saved!
====== 54 epoch of 100 ======

====== 100 Step of 196 ======
Train Acc : 0.7033570544554455
Train Loss : 0.0037958493921905756
Valid Acc : 0.7011
Valid Loss : 0.003429016331210